In [6]:
from pandana.loaders import osm
import folium, requests, json
import pandas as pd
from config import gkey

In [2]:
df = pd.read_csv('Neighborhoods__Regions_ (1).csv')

In [3]:
df

,OBJECTID,NAME,COMMPLAN,SHARED,COALIT,HORZ_VERT,MAPLABEL,ID,Shape_Length,Shape_Area
0,1,CATHEDRAL PARK,,,NPNS,HORZ,Cathedral Park,31,11434.254777,5.424298e+06
1,2,UNIVERSITY PARK,,,NPNS,HORZ,University Park,88,11950.859827,6.981457e+06
2,3,PIEDMONT,ALBINA,,NPNS,VERT,Piedmont,70,10849.327392,6.079530e+06
3,4,WOODLAWN,ALBINA,,NECN,HORZ,Woodlawn,93,8078.360994,3.870554e+06
4,5,CULLY ASSOCIATION OF NEIGHBORS,,,CNN,HORZ,Cully Association of Neighbors,23,18179.392090,1.658062e+07
...,...,...,...,...,...,...,...,...,...,...
93,94,SUNDERLAND ASSOCIATION OF NEIGHBORS,,,CNN,HORZ,Sunderland Association of Neighbors,85,20706.496916,9.322958e+06
94,95,PORTSMOUTH,,,NPNS,HORZ,Portsmouth,18,10003.795565,5.722844e+06
95,96,HAYDEN ISLAND NEIGHBORHOOD NETWORK,,,NPNS,HORZ,Hayden Island Neighborhood Network,35,27375.215778,1.579258e+07
96,97,BRIDGETON,,,NPNS,HORZ,Bridgeton,11,8635.720662,1.433719e+06


In [4]:
m = folium.Map([45.505, -122.675], tiles='cartodbpositron')

url = 'https://opendata.arcgis.com/datasets/9f50a605cf4945259b983fa35c993fe9_125.geojson'

gj = folium.GeoJson(
    url,
    name='geojson'
).add_to(m)

In [5]:
m

In [9]:
geo = requests.get(url).json()
print(json.dumps(geo, indent=4))

{
    "type": "FeatureCollection",
    "name": "Neighborhoods__Regions_",
    "crs": {
        "type": "name",
        "properties": {
            "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
        }
    },
    "features": [
        {
            "type": "Feature",
            "properties": {
                "OBJECTID": 1,
                "NAME": "CATHEDRAL PARK",
                "COMMPLAN": " ",
                "SHARED": " ",
                "COALIT": "NPNS",
                "HORZ_VERT": "HORZ",
                "MAPLABEL": "Cathedral Park",
                "ID": 31,
                "Shape_Length": 11434.254777015894,
                "Shape_Area": 5424297.812272152
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            -122.74774080646131,
                            45.57641768898887
                        ],
                        [
                        

In [17]:
names = []
lats = []
lngs = []
for i in geo['features']:
    names.append(i['properties']['NAME'])
    lats.append(i['geometry']['coordinates'][0][0][1])
    lngs.append(i['geometry']['coordinates'][0][0][0])
query_df = pd.DataFrame()
query_df['name'] = names
query_df['lat'] = lats
query_df['lng'] = lngs

query_df
pd.set_option('display.max_rows',None)


In [25]:
maps_base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
zipcodes = []
for index, row in query_df.iterrows():
    lat = row.lat
    lng = row.lng
    name = row.name
    
    response = requests.get(f"{maps_base_url}latlng={lat},{lng}&key={gkey}").json()
#     print(len(response['results'][0]['address_components']))
    if response['results'][0]['address_components'][-1]['types'][0] == 'postal_code':
        zipcodes.append(response['results'][0]['address_components'][-1]['long_name'])
    elif response['results'][0]['address_components'][-2]['types'][0] == 'postal_code':
        zipcodes.append(response['results'][0]['address_components'][-2]['long_name'])
    else:
        zipcodes.append(response['results'][0]['address_components'][-3]['long_name'])

query_df['Zipcode'] = zipcodes

In [26]:
query_df = query_df.loc[query_df['name'] != 'MC UNCLAIMED #13', :]
query_df = query_df.loc[query_df['name'] != 'CRESTWOOD', :]

In [27]:
query_df

,name,lat,lng,Zipcode
0,CATHEDRAL PARK,45.5764,-122.748,97210
1,UNIVERSITY PARK,45.575,-122.708,97217
2,PIEDMONT,45.5866,-122.675,97217
3,WOODLAWN,45.5663,-122.644,97211
4,CULLY ASSOCIATION OF NEIGHBORS,45.5483,-122.615,97213
5,ARBOR LODGE,45.5772,-122.679,97217
6,OVERLOOK,45.5546,-122.679,97217
7,CONCORDIA,45.5554,-122.642,97211
8,PARKROSE,45.5554,-122.568,97220
9,SUMNER ASSOCIATION OF NEIGHBORS,45.5554,-122.568,97220
